# Imports

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim 
import matplotlib.pyplot as plt
from torchvision import datasets
from torchvision.transforms import ToTensor

# Datasets Loading

In [3]:
train_dataset = datasets.MNIST(
    root='data',
    train=True,
    download=True,
    transform=ToTensor()
)

test_dataset = datasets.MNIST(
    root='data',
    train=False,
    download=True,
    transform=ToTensor()
)

In [4]:
print(train_dataset.data.shape)
print(train_dataset.targets.shape)

torch.Size([60000, 28, 28])
torch.Size([60000])


In [5]:
train_loader = torch.utils.data.DataLoader(
    dataset=train_dataset,
    batch_size=100,
    shuffle=True
)

test_loader = torch.utils.data.DataLoader(
    dataset=test_dataset,
    batch_size=100,
    shuffle=True
)

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Model Creation (CNN)

In [7]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 3 * 3, 64)
        self.fc2 = nn.Linear(64, 10)
    
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = F.relu(self.conv3(x))
        x = torch.flatten(x, 1) # x.shape = N, 576 | (64 * 3 * 3)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = CNN().to(device)

In [8]:
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

In [9]:
model.train()

for epoch in range(2):
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)

        # Forward pass
        predictions = model(data)
        loss = loss_fn(predictions, target)

        # Backward and optimize
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if (batch_idx + 1) % 100 == 0:
            print (f'Epoch [{epoch + 1}], Step [{batch_idx + 1}], Loss: {loss.item():.4f}')

Epoch [1], Step [100], Loss: 2.0904
Epoch [1], Step [200], Loss: 0.9512
Epoch [1], Step [300], Loss: 0.5395
Epoch [1], Step [400], Loss: 0.5610
Epoch [1], Step [500], Loss: 0.4042
Epoch [1], Step [600], Loss: 0.2300
Epoch [2], Step [100], Loss: 0.4318
Epoch [2], Step [200], Loss: 0.3356
Epoch [2], Step [300], Loss: 0.2351
Epoch [2], Step [400], Loss: 0.4813
Epoch [2], Step [500], Loss: 0.2302
Epoch [2], Step [600], Loss: 0.3248


In [11]:
model.eval()

with torch.no_grad():
    n_correct = 0
    n_samples = len(test_loader.dataset)

    for data, targets in test_loader:
        data, targets = data.to(device), targets.to(device)

        output = model(data)
        _, predicted = torch.max(output, 1)

        n_correct += (predicted == targets).sum().item()

    accuracy = 100.0 * n_correct / n_samples
    print(f'Accuracy of the CNN: {accuracy} %')

Accuracy of the CNN: 92.59 %
